In [3]:
!pip3 install diffusers
!pip3 install torch torchvision
!pip3 install PyPDF2
!pip3 install spacy
!python -m spacy download en_core_web_sm
!pip3 install nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [4]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import spacy
nlp = spacy.load("en_core_web_sm")

In [84]:
import PyPDF2

def extract_chapters(pdf_path, start_page=None,end_page=None):
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)

       
        if start_page is None:
            start_page = 1
        if end_page is None:
            end_page = len(pdf_reader.pages)

        start_page = max(1, min(len(pdf_reader.pages), start_page))
        end_page = max(start_page, min(len(pdf_reader.pages), end_page))

        text = ""

        for page_number in range(start_page - 1, end_page):
            page = pdf_reader.pages[page_number]
            text += page.extract_text()
    extracted_text=''
    for token in text.split():
        extracted_text+=token+' '
    return extracted_text




In [85]:
def coreference_resolution(text):
    nlp = spacy.load('en_core_web_trf')
    nlp.add_pipe('coreferee')
    doc = nlp(text)
    # doc._.coref_chains.print()
    # print(doc._.coref_chains.resolve(doc[10]))
    input_text=[]
    for token in doc:
        input_text.append(token)
    # print(input_text)
    chains=doc._.coref_chains
    for c in chains:
        for i in range(len(c)):
            index=c[i][0]
            coref_noun=doc._.coref_chains.resolve(doc[index])
            if coref_noun is not None:
                input_text[index]=coref_noun[0]
    resolved_text=""
    for token in input_text:
        resolved_text+=str(token)+" "
    return resolved_text

In [86]:
# create chunks of n sentences each

def chunk_text(text, n):
    about_doc = nlp(text)
    sentences = list(about_doc.sents)
    chunks = [sentences[i:i + n] for i in range(0, len(sentences), n)]
    chunked_text = [' '.join(map(str, chunk)) for chunk in chunks]
    return chunked_text


In [87]:
def extract_information(text):
    doc = nlp(text)
    # Extract relevant information (nouns, adjectives, verbs, proper nouns, and words describing position)
    relevant_info = [
        token.text 
        for token in doc 
        if token.pos_ in ['NOUN', 'ADJ', 'VERB', 'PROPN'] or token.dep_ in ['prep', 'pobj']
    ]
    text=' '.join(relevant_info)
    text = text.replace('- ', '')
    return text


In [93]:
def remove_punctuation(text):
    PUNCT_TO_REMOVE = string.punctuation
    return ' '.join(text.translate(str.maketrans('', '', PUNCT_TO_REMOVE)).split())
def lemmatize_words(text):
    lemmatizer = WordNetLemmatizer()
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
def lowercase_text(text):
    return text.lower()
def remove_single_alpha(text):
    return " ".join([word for word in text.split() if len(word)>1 and word.isalpha()])

In [99]:

def preprocess(pdf_path,start_page,end_page,n_sentences_per_chunk):
    text=extract_chapters(pdf_path,start_page,end_page)
    resolved_text=coreference_resolution(text)
    # print(resolved_text)
    chunks = chunk_text(resolved_text, n_sentences_per_chunk)
    preprocessed_text=[]
    for chunk in chunks:
        temp=extract_information(chunk)
        temp=remove_punctuation(temp)
        temp=lemmatize_words(temp)
        temp=lowercase_text(temp)
        temp=remove_single_alpha(temp)
        preprocessed_text.append(temp)
    return preprocessed_text
        

    

In [100]:
pdf_path = 'sample.pdf'
start_page = 13    #set none to process whole pdf
end_page = 14
n_sentences_per_chunk = 5
preprocessed_text=preprocess(pdf_path,start_page,end_page,n_sentences_per_chunk)
for chunk in preprocessed_text:
    print(chunk)
    print('*'*100)

study inscarlet chapter mr sherlock holmes in year took degree of doctor of medicine of university of london proceeded to netley go through course prescribed for surgeon in army completed study attached to fifth northumberland fusiliers assistant surgeon regiment stationed in india at time join war second afghan war had
****************************************************************************************************
on landing at bombay learned corp advanced through pass in enemy country followed with many other ofﬁcers in same situation myself succeeded in reaching candahar in safety found regiment at entered upon new duty campaign brought honour promotion many for me campaign had but misfortune disaster removed from brigade attached to berkshires with whom served at fatal battle of maiwand struck on shoulder jezail bullet shattered bone grazed subclavian artery
****************************************************************************************************
fallen into hand of 